In [16]:
# Import necessary modules
# glob allows unix style pathname pattern expansion
from glob import glob
# pydicom is the python dicom reader
import pydicom as dicom
import nibabel.nicom.csareader as csareader
import os
import pandas as pd

In [2]:
# 5-T1w_MPR_vNav -- MPRAGE structrual dicoms
# 6-fMRI_DistortionMap_PA -- fmri field map dicoms
# 7-fMRI_DistortionMap_AP -- fmri field map dicoms opposite phase encode direction
# 9-fMRI_REVL_ROI_loc_2 -- fmri localizer dicoms run1
# 10-fMRI_REVL_Study_1 -- fmri task dicoms run1
# 14-dMRI_DistortionMap_AP_dMRI_REVL -- dwi field map dicoms
# 16-dMRI_AP_REVL -- diffusion weighted dicoms

# directories will have to be specific to your computer
base_dir = '/Users/amattfel/Documents/GitHub'
proj_dir = '/mattfeld_2020/sourcedata'
subj_dir = '/Mattfeld_REVL-000-vCAT-021-S1/scans/10-fMRI_REVL_Study_1/resources/DICOM/files'

# with strings I can concatenate them with simple addition lines
data_dir = base_dir + proj_dir + subj_dir

# Here I use glob to grab the dicom files
# Why would I use glob?  What does it give me?
data_files = glob(data_dir + '/*')


In [3]:
data_files[0:5]

['/Users/amattfel/Documents/GitHub/mattfeld_2020/sourcedata/Mattfeld_REVL-000-vCAT-021-S1/scans/10-fMRI_REVL_Study_1/resources/DICOM/files/1.3.12.2.1107.5.2.43.166003.30000019061912563618000000017-10-6-1j8kpi7.dcm',
 '/Users/amattfel/Documents/GitHub/mattfeld_2020/sourcedata/Mattfeld_REVL-000-vCAT-021-S1/scans/10-fMRI_REVL_Study_1/resources/DICOM/files/1.3.12.2.1107.5.2.43.166003.30000019061912563618000000017-10-292-x6mb7t.dcm',
 '/Users/amattfel/Documents/GitHub/mattfeld_2020/sourcedata/Mattfeld_REVL-000-vCAT-021-S1/scans/10-fMRI_REVL_Study_1/resources/DICOM/files/1.3.12.2.1107.5.2.43.166003.30000019061912563618000000017-10-288-v6og2m.dcm',
 '/Users/amattfel/Documents/GitHub/mattfeld_2020/sourcedata/Mattfeld_REVL-000-vCAT-021-S1/scans/10-fMRI_REVL_Study_1/resources/DICOM/files/1.3.12.2.1107.5.2.43.166003.30000019061912563618000000017-10-33-kymxgv.dcm',
 '/Users/amattfel/Documents/GitHub/mattfeld_2020/sourcedata/Mattfeld_REVL-000-vCAT-021-S1/scans/10-fMRI_REVL_Study_1/resources/DICOM/f

In [4]:
# Here I am using the python dicom reader to read in a dicom header
# What should go between the square brackets []
data_set = dicom.dcmread(data_files[0])

In [6]:
data_set

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 204
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.2.43.166003.2019061915480017886637273
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.40.0.13.1.1
(0002, 0013) Implementation Version Name         SH: 'dcm4che-2.0'
(0002, 0016) Source Application Entity Title     AE: 'AN_MEDCOMNT204'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M', 'ND', 'MOSAIC']
(0008, 0012) Instance Creation Date              DA: '20190619'
(0008, 0013) Instance Creation Time              TM: '154835.406000'
(0008, 0016) SOP Cl

In [11]:
# Elements we're interested in
# Repetition Time
# Echo Time
# Acquisition Matrix 
# Flip Angle
# Acquisition Number
# Protocol Name
# Slice times = data_set[0x00191029].value

print('The number of TRs collected equals: {0}'.format(len(data_files)))
print('The timing of the TR was: {0} ms'.format(data_set.RepetitionTime))
print('The timing of the TE was: {0} ms'.format(data_set.EchoTime))
print('The flip angle in degrees was: {0}'.format(data_set.FlipAngle))
print('The name of the scan was: {0}'.format(data_set.FlipAngle))
print('The phase encode direction was: {0}'.format(data_set[int('00181312', 16)].value))
print('COL = AP or PA; ROW = RL or LR')
print('The slice timing was: {0}'.format(data_set[0x00191029].value))
print('The field of view - FOV was: {0}'.format(data_set[int('0051100c', 16)].value))

# Why use the following format to access information in the header?
#print(data_set[int('0051100c', 16)].value) # data_set[0x0051100c].value
# Allows us to access information that is private in the dicom header.

# How do I find phase encoding direction information?

csa_str = data_set[int('00291010', 16)].value
csa_tr = csareader.read(csa_str)
print('The phase encode direction was: {0}'.format(csa_tr['tags']['PhaseEncodingDirectionPositive']['items'][0]))
print('1 = -; 0 = +')
print('COL, 1 = PA; COL, 0 = AP')



The number of TRs collected equals: 355
The timing of the TR was: 1760 ms
The timing of the TE was: 35 ms
The flip angle in degrees was: 52
The name of the scan was: 52
The phase encode direction was: COL
COL = AP or PA; ROW = RL or LR
The slice timing was: [1265.00000001, 0.0, 870.0, 77.50000001, 947.50000001, 157.5, 1027.5, 237.50000002, 1107.50000001, 315.0, 1185.0, 475.00000001, 1345.0, 552.49999999, 1422.50000002, 632.50000001, 1502.5, 709.99999999, 1580.00000002, 790.00000001, 1660.0, 394.99999999, 1265.00000001, 0.0, 870.0, 77.50000001, 947.50000001, 157.5, 1027.5, 237.50000002, 1107.50000001, 315.0, 1185.0, 475.00000001, 1345.0, 552.49999999, 1422.50000002, 632.50000001, 1502.5, 709.99999999, 1580.00000002, 790.00000001, 1660.0, 394.99999999, 1265.00000001, 0.0, 870.0, 77.50000001, 947.50000001, 157.5, 1027.5, 237.50000002, 1107.50000001, 315.0, 1185.0, 475.00000001, 1345.0, 552.49999999, 1422.50000002, 632.50000001, 1502.5, 709.99999999, 1580.00000002, 790.00000001, 1660.0, 39

# Using heudiconv to convert dicoms into BIDs format

In [19]:
# Step 1 - run heudiconv
#heudiconv -d '../sourcedata/Mattfeld_REVL-000-vCAT-{subject}-S1/*/*/*/*/*/*' -s 021 -c none -f convertall -o /Users/amattfel/Documents/GitHub/mattfeld_2020/dset

#/Users/amattfel/Documents/GitHub/mattfeld_2020/dset/.heudiconv/021/info
path2info_file = os.path.join(base_dir, 'mattfeld_2020', 'dset', '.heudiconv/021/info')
dicominfo_df = pd.read_csv(path2info_file + '/dicominfo.tsv', sep="\t")
dicominfo_df.head()

,total_files_till_now,example_dcm_file,series_id,dcm_dir_name,series_files,unspecified,dim1,dim2,dim3,dim4,...,study_description,referring_physician_name,series_description,sequence_name,image_type,accession_number,patient_age,patient_sex,date,series_uid
0,3,1.3.12.2.1107.5.2.43.166003.300000190619125636...,1-localizer_32ch,files,3,NaN,512,512,3,1,...,Mattfeld_REVL,NaN,localizer_32ch,*fl2d1,"('ORIGINAL', 'PRIMARY', 'M', 'ND', 'NORM')",NaN,042Y,M,20190619,1.3.12.2.1107.5.2.43.166003.201906191516097435...
1,35,1.3.12.2.1107.5.2.43.166003.300000190619125636...,2-T1w_MPR_vNav_setter,files,32,NaN,32,32,32,1,...,Mattfeld_REVL,NaN,T1w_MPR_vNav_setter,ABCD3d1_32ns,"('ORIGINAL', 'PRIMARY', 'M', 'ND')",NaN,042Y,M,20190619,1.3.12.2.1107.5.2.43.166003.201906191517057627...
2,180,1.3.12.2.1107.5.2.43.166003.300000190619125636...,3-T1w_MPR_vNav,files,145,NaN,32,32,32,145,...,Mattfeld_REVL,NaN,T1w_MPR_vNav_setter,ABCD3d1_32ns,"('ORIGINAL', 'PRIMARY', 'M', 'ND', 'MOSAIC')",NaN,042Y,M,20190619,1.3.12.2.1107.5.2.43.166003.201906191517406872...
3,356,1.3.12.2.1107.5.2.43.166003.300000190619125636...,4-T1w_MPR_vNav,files,176,NaN,256,256,176,1,...,Mattfeld_REVL,NaN,T1w_MPR_vNav,tfl3d1_16ns,"('ORIGINAL', 'PRIMARY', 'M', 'ND')",NaN,042Y,M,20190619,1.3.12.2.1107.5.2.43.166003.201906191524081007...
4,532,1.3.12.2.1107.5.2.43.166003.300000190619125636...,5-T1w_MPR_vNav,files,176,NaN,256,256,176,1,...,Mattfeld_REVL,NaN,T1w_MPR_vNav,tfl3d1_16ns,"('ORIGINAL', 'PRIMARY', 'M', 'ND', 'NORM')",NaN,042Y,M,20190619,1.3.12.2.1107.5.2.43.166003.201906191517406926...
